In [1]:
#include <iostream>
#include <string>
#include <math.h>       /* cosh, log */
using namespace std;

In [2]:
/*Cargo la librería de Delphes*/
//gSystem->Load("/home/camilo/Proyectos/Delphes/delphes/libDelphes.so");
//gSystem->AddDynamicPath("/scratch/cms/Delphes/delphes/");
//gSystem->Load("libDelphes");

In [3]:
/*Instancio un objeto TChain*/
TChain chain("Delphes");

In [4]:
/*Creo las variables necesarias para el for*/
std::string nom1="/Storage/scratch/camilo/Proyectos/MadGraph/Datos/Signal_2j_mu-nu/Events/run_";
std::string nom2="/tag_1_delphes_events.root";
std::string num, out;

In [5]:
/*Llenado del chain con todos los runs*/
for (Int_t i=1; i<11; i=i+1 ) {
   num = std::to_string(i);
   if( i < 10 ) {
       out= nom1+"0"+num+nom2;
    }
//convierto en cadena de caracteres constante (es lo que recibe la función)
    const char * c = out.c_str();
/*pego todos los datos*/
    chain.Add(c);
}

In [6]:
//Creo un objeto de treereader, que recive la dirección en memoria de
//la cadena (TChain) como argumento
ExRootTreeReader *treeReader = new ExRootTreeReader(&chain);

input_line_38:4:36: error: unknown type name 'ExRootTreeReader'
ExRootTreeReader *treeReader = new ExRootTreeReader(&chain);
                                   ^


In [7]:
/*Leo el numero de entradas*/
Long64_t numberOfEntries = treeReader->GetEntries()

input_line_40:2:3: error: use of undeclared identifier 'treeReader'
 (treeReader->GetEntries())
  ^
Error in <HandleInterpreterException>: Error evaluating expression (treeReader->GetEntries()).
Execution of your code was aborted.


In [8]:
/*Obtengo las ramas que deseo*/
TClonesArray *branchMuon = treeReader->UseBranch("Muon");
TClonesArray *branchMissingET = treeReader->UseBranch("MissingET");
TClonesArray *branchJet = treeReader->UseBranch("Jet");

input_line_42:2:3: error: use of undeclared identifier 'treeReader'
 (treeReader->UseBranch("Muon"))
  ^
Error in <HandleInterpreterException>: Error evaluating expression (treeReader->UseBranch("Muon")).
Execution of your code was aborted.


In [9]:
// Inicializo los contadores
Int_t cut1=0, cut2=0, cut3=0, cut4=0, cut5=0;

In [10]:
for(Int_t entry = 0; entry < numberOfEntries; ++entry)
    {
        // Se carga un evento espesifico para ser analizado
        treeReader->ReadEntry(entry);

        // Se hace el primer corte, se pide que tenga al menos dos jets
        if(branchJet->GetEntries() > 1)
        {
            ++cut1; //Cuantos eventos pasan el corte 1
            
            Int_t count=0;
            //Se cuentan los jets tageados como b
            for (Int_t i=0; i<branchJet->GetEntries(); ++i)
            {
                Jet* jet = (Jet*) branchJet->At(i);
                if(jet->BTag>0)
                {
                    ++count;
                }
    
            }

            if( count>1 )
            {
                ++cut2; //Cuantos eventos pasan el corte 2
                //Inicializo los obejos clase muon
                Muon *muon1, *muon2;

                // Para ser contados deben de tener al menos 2 muones
                if(branchMuon->GetEntries() > 1)
                {
                    ++cut3; //Cuantos eventos pasan el corte 3
                    // Take first two electrons, tomo solo dos porque estan organizados por PT
                    muon1 = (Muon *) branchMuon->At(0);
                    muon2 = (Muon *) branchMuon->At(1);
                    //Si la carga es diferente
                    int charge = muon1->Charge * muon2->Charge;
                    if( charge < 0 )
                    {
                        ++cut4; //Cuantos eventos pasan el corte 4
                
                        double MassMuon=(muon1->P4()+muon2->P4()).M();
                        if( !((MassMuon>80)&&(MassMuon<100)) )
                        {
                            ++cut5; //Cuantos eventos pasan el corte 4
                     
                            
                            // Plot jet transverse momentum
                            //histJetPT->Fill(jet->PT);
                            // Plot their invariant mass
                            //histMass->Fill(((elec1->P4()) + (elec2->P4())).M());
               
                        
                        
                        }
                    }
                }
            }
        }
    }

input_line_44:16:33: error: expected expression
                Jet* jet = (Jet*) branchJet->At(i);
                                ^
input_line_44:17:20: error: use of undeclared identifier 'jet'
                if(jet->BTag>0)
                   ^
input_line_44:28:17: error: unknown type name 'Muon'
                Muon *muon1, *muon2;
                ^
input_line_44:35:30: error: use of undeclared identifier 'Muon'
                    muon1 = (Muon *) branchMuon->At(0);
                             ^
input_line_44:35:36: error: expected expression
                    muon1 = (Muon *) branchMuon->At(0);
                                   ^
input_line_44:36:21: error: use of undeclared identifier 'muon2'
                    muon2 = (Muon *) branchMuon->At(1);
                    ^
input_line_44:36:30: error: use of undeclared identifier 'Muon'
                    muon2 = (Muon *) branchMuon->At(1);
                             ^
input_line_44:36:36: error: expected expression
        

In [11]:
cout<<"#Raw ="<<numberOfEntries<<endl;
cout<<"#Cut1="<<cut1<<endl;
cout<<"#Cut2="<<cut2<<endl;
cout<<"#Cut3="<<cut3<<endl;
cout<<"#Cut4="<<cut4<<endl;
cout<<"#Cut5="<<cut5<<endl<<endl;


float EFFN1=(float)cut1/(float)numberOfEntries;
float EFFN2=(float)cut2/(float)cut1;
float EFFN3=(float)cut3/(float)cut2;
float EFFN4=(float)cut4/(float)cut3;
float EFFN5=(float)cut5/(float)cut4;


float EFFA1=(float)cut1/(float)numberOfEntries;
float EFFA2=(float)cut2/(float)numberOfEntries;
float EFFA3=(float)cut3/(float)numberOfEntries;
float EFFA4=(float)cut4/(float)numberOfEntries;
float EFFA5=(float)cut5/(float)numberOfEntries;

cout<<"EFFA1="<<EFFA1<<endl;
cout<<"EFFA2="<<EFFA2<<endl;
cout<<"EFFA3="<<EFFA3<<endl;
cout<<"EFFA4="<<EFFA4<<endl;
cout<<"EFFA5="<<EFFA5<<endl<<endl;


cout<<"EFFN1="<<EFFN1<<endl;
cout<<"EFFN2="<<EFFN2<<endl;
cout<<"EFFN3="<<EFFN3<<endl;
cout<<"EFFN4="<<EFFN4<<endl;
cout<<"EFFN5="<<EFFN5<<endl;

#Raw =0
#Cut1=0
#Cut2=0
#Cut3=0
#Cut4=0
#Cut5=0

EFFA1=-nan
EFFA2=-nan
EFFA3=-nan
EFFA4=-nan
EFFA5=-nan

EFFN1=-nan
EFFN2=-nan
EFFN3=-nan
EFFN4=-nan
EFFN5=-nan


In [12]:
355148
341827
126190
55688
55687
48077

input_line_46:2:8: error: expected ';' after expression
 355148
       ^
       ;
input_line_46:3:7: error: expected ';' after expression
341827
      ^
      ;
input_line_46:4:7: error: expected ';' after expression
126190
      ^
      ;
input_line_46:5:6: error: expected ';' after expression
55688
     ^
     ;
input_line_46:6:6: error: expected ';' after expression
55687
     ^
     ;
